In [97]:
import pandas as pd
DelDF = pd.read_csv('Deliveries.csv')
OldDelDF = pd.read_csv('DeliveriesOld.csv')
DaysDF = pd.read_csv('Days.csv')
WeeksDF = pd.read_csv('Weeks.csv')
gpsDF = pd.read_csv('GPS.csv')

In [2]:
OldDelDF.head()

,ID,Date,Restaurant Name,Total,Pay,Tip,Peak Bonus,Stacked,Version
0,2,3/27,Erika's Restaurant,$6.75,$5.25,$1.50,$0.00,NaN,1.0
1,3,3/27,Chick-fil-A,$8.00,$4.50,$3.50,$0.00,NaN,1.0
2,4,3/27,Wendy's,$7.00,$3.00,$4.00,$0.00,NaN,1.0
3,5,3/27,Annam Pho and Sandwiches,$7.00,$7.00,$0.00,$0.00,NaN,1.0
4,6,3/31,Disfruta,$4.75,$3.75,$1.00,$0.00,NaN,1.0


In [3]:
print(str(OldDelDF["Restaurant Name"].nunique()) + " " + str(DelDF["Restaurant Name"].nunique()))

132 48


In [4]:
frames = [OldDelDF, DelDF]
result = pd.concat(frames)

In [5]:
result["Restaurant Name"].nunique()

141

In [6]:
result["Pay"] = result["Pay"].str.replace('$', '')
result["Tip"] = result["Tip"].str.replace('$', '')
result["Total"] = result["Total"].str.replace('$', '')
result["Peak Bonus"] = result["Peak Bonus"].str.replace('$', '')

/tmp/ipykernel_15199/1212687638.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Pay"] = result["Pay"].str.replace('$', '')
/tmp/ipykernel_15199/1212687638.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Tip"] = result["Tip"].str.replace('$', '')
/tmp/ipykernel_15199/1212687638.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  result["Total"] = result["Total"].str.replace('$', '')
/tmp/ipykernel_15199/1212687638.py:4: FutureWarning: The default value of regex will change from True to False in a f

In [7]:
result.head()

,ID,Date,Restaurant Name,Total,Pay,Tip,Peak Bonus,Stacked,Version,Start Time,Miles,End Time,Est. Duration
0,2,3/27,Erika's Restaurant,6.75,5.25,1.50,0.00,NaN,1.0,NaN,NaN,NaN,NaN
1,3,3/27,Chick-fil-A,8.00,4.50,3.50,0.00,NaN,1.0,NaN,NaN,NaN,NaN
2,4,3/27,Wendy's,7.00,3.00,4.00,0.00,NaN,1.0,NaN,NaN,NaN,NaN
3,5,3/27,Annam Pho and Sandwiches,7.00,7.00,0.00,0.00,NaN,1.0,NaN,NaN,NaN,NaN
4,6,3/31,Disfruta,4.75,3.75,1.00,0.00,NaN,1.0,NaN,NaN,NaN,NaN


In [8]:
result["Version"].unique()

array([ 1., nan,  2.,  3.])

In [9]:
result.dtypes

ID                   int64
Date                object
Restaurant Name     object
Total               object
Pay                 object
Tip                 object
Peak Bonus          object
Stacked             object
Version            float64
Start Time          object
Miles               object
End Time            object
Est. Duration       object
dtype: object

In [10]:
result["Total"] = result["Total"].astype('float')
result["Pay"] = result["Pay"].astype('float')
result["Tip"] = result["Tip"].astype('float')
result["Peak Bonus"] = result["Peak Bonus"].astype('float')

In [11]:
print(result.shape)
result1 = result[result["Version"] == 1.0]
print(result1.shape)
result2 = result[result["Version"] == 2.0]
print(result2.shape)
result3 = result[result["Version"] == 3.0]
print(result3.shape)

(674, 13)
(370, 13)
(202, 13)
(101, 13)


In [12]:
resultAVG = result.groupby(["Restaurant Name"]).agg({"Pay": "mean", "Tip": "mean"})
resultSUM = result.groupby(["Restaurant Name"]).agg({"Pay": "sum", "Tip": "sum"})
#Pay": "sum", "Tip": "sum", 

In [13]:
resultAVG = resultAVG.round({'Pay': 2, 'Tip': 2})
resultSUM = resultSUM.round({'Pay': 2, 'Tip': 2})

In [14]:
resultAVG = resultAVG.sort_values("Tip", ascending=False)
resultAVG.head(20)

,Pay,Tip
Restaurant Name,,
Chef Tang,14.00,13.00
Killer Pizza From Mars,4.25,12.50
Albertsons,4.58,12.39
Brooklyn Boyz Pizza,5.25,12.15
Sabai Sabai Thai Kitchen,8.25,12.00
Hunter Steakhouse,10.00,11.50
Ceja's Mexican Diner & Grill,5.50,10.50
Mangia e Bevi,6.75,10.00
L&L Hawaiian Barbecue,7.75,10.00


In [15]:
resultSUM.sort_values("Pay", ascending=False)

,Pay,Tip
Restaurant Name,,
Chick-fil-A,428.25,272.01
Taco Bell,216.87,158.70
McDonald's,207.50,163.50
Dave's Hot Chicken,142.25,133.00
Chipotle,131.25,107.38
...,...,...
Umi Japanese Grill & Sushi,3.25,9.50
Pita Guys,3.25,4.00
Pick Up Stix(SOF),3.25,2.02


In [16]:
resultSUM.sort_values("Tip", ascending=False)

,Pay,Tip
Restaurant Name,,
Chick-fil-A,428.25,272.01
McDonald's,207.50,163.50
Taco Bell,216.87,158.70
Dave's Hot Chicken,142.25,133.00
Chipotle,131.25,107.38
...,...,...
Quik Wok,6.50,0.00
Guanatos City Mexican Restaurant,4.00,0.00
TenderFix by Noah Schnapp,6.25,0.00


In [17]:
sorted(result["Restaurant Name"].unique())

['7-Eleven',
 'Acai Superfruits',
 'Advanced Auto Parts',
 'Albertsons',
 "Anita's Mexican Restaurant & Cantina",
 'Annam Pho and Sandwiches',
 "Applebee's",
 "Applebee's Grill & Bar",
 'Beach Hut Deli',
 'Bloom Haus',
 'Board & Brew',
 'Board and Brew',
 'Broken Yolk',
 'Brooklyn Boyz Pizza',
 'Buffalo Wild Wings',
 'Burger King',
 'Cafe de Thai & Sushi',
 "Carl's Jr.",
 "Ceja's Mexican Diner & Grill",
 'Chef Tang',
 'Chick',
 'Chick-fil-A',
 "Chili's Grill & Bar",
 "Chin's Szechwan",
 'China Fusion',
 'Chipotle',
 'Colimas Mexican Food',
 'Corner Grill',
 'Cornerstone Network, Inc.',
 'Cream Of The Crop',
 'Crumbl Cookies',
 "Cusimano's Pizzeria",
 'Dashmart',
 "Dave's Hot Chicken",
 "Deiberto's Mexican Food",
 'Del Taco',
 "Denny's",
 'Ding Tea',
 'Disfruta',
 'El Bronco',
 'El Pollo Loco',
 "Enzo's BBQ Ale House",
 'Epic Wings',
 "Eriberto's Mexican Food",
 "Erika's Restaurant",
 'Everbowl',
 'FTD',
 "Felix's BBQ With Soul",
 'Firehouse Subs',
 "Gianni's Pizza",
 'GoGo Japan - Sush

In [18]:
#Fix typos, for places with two names use whichever one is most recent

#Check the the's
## The Cheesecake Factory, The Cheesesteak Grill
# The Draft Restaurant & Sports bar, The Habit Burger Grill, The Meltdown, The Poke Cafe

#Things to fix:
#Applebee's || Applebee's Grill & Bar
#Board & Brew || Board and Brew
#Chick || Chick-fil-A
#Cornerstone Network, Inc. || Burger King
#Jersey Mike's || Jersey Mike's Subs
#Little Caesars || Little Caesars Pizza
#Maan's Mediterranean Grill || Maans Mediterranean Grill
#Pick Up Stix || Pick Up Stix(SOF)
#Sonic Drive-In || Sonic Drive-in
#Thai Thai Oceanside Restaurant || Thai Thai Oceanside restaurant
#The Privateer Coal Fire Pizz || The Privateer Coal Fire Pizza
#Two Brothers From Italy || Two Brothers From Italy Pizza
#Wingstop || Wingstop Drive

In [22]:
numUntipped = (result["Tip"] == 0.0).sum()
print(numUntipped)
numTipped = (result["Tip"] != 0.0).sum()
print(numTipped)
percTipped = (numTipped/(numUntipped+numTipped))
print(percTipped)

77
597
0.8857566765578635


In [ ]:
# 88.58% of customers tip

In [43]:
#Doodle that converts 'Date' attribute to Timestamp object
#Might be easiest way to extract day of the week from the string given in 'Date'

from datetime import date, time

datevar = result.iloc[600]["Date"]
timevar = result.iloc[600]["Start Time"]
print(datevar, " ", timevar)
datevar1 = '2023/' + datevar + " " + timevar
datevar1 = pd.Timestamp(datevar1)
print(datevar1)
print(datevar1.day_name())
print(type(datevar1))

5/27   7:19:00 PM
2023-05-27 19:19:00
Saturday
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [44]:
result.head()

,ID,Date,Restaurant Name,Total,Pay,Tip,Peak Bonus,Stacked,Version,Start Time,Miles,End Time,Est. Duration
0,2,3/27,Erika's Restaurant,6.75,5.25,1.5,0.0,NaN,1.0,NaN,NaN,NaN,NaN
1,3,3/27,Chick-fil-A,8.00,4.50,3.5,0.0,NaN,1.0,NaN,NaN,NaN,NaN
2,4,3/27,Wendy's,7.00,3.00,4.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN
3,5,3/27,Annam Pho and Sandwiches,7.00,7.00,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN
4,6,3/31,Disfruta,4.75,3.75,1.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN


[2.0, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0, 5.25, 5.27, 5.37, 5.38, 5.5, 5.75, 5.88, 6.0, 6.03, 6.25, 6.5, 6.57, 6.75, 7.0, 7.25, 7.5, 7.62, 7.7, 7.75, 7.94, 8.0, 8.25, 8.5, 8.56, 8.75, 9.0, 9.07, 9.25, 9.5, 9.54, 9.63, 9.7, 9.75, 9.83, 10.0, 10.03, 10.23, 10.25, 10.38, 10.49, 10.5, 10.75, 10.83, 10.94, 11.0, 11.25, 11.5, 11.63, 11.75, 12.0, 12.25, 12.36, 12.5, 12.51, 12.63, 12.75, 13.0, 13.25, 13.5, 13.75, 14.0, 14.01, 14.23, 14.25, 14.5, 14.62, 14.64, 14.75, 15.0, 15.25, 15.5, 15.75, 16.0, 16.25, 16.5, 16.64, 16.75, 16.87, 17.0, 17.25, 17.4, 17.5, 17.75, 18.0, 18.25, 18.75, 19.0, 19.5, 19.75, 20.16, 20.25, 20.5, 22.0, 23.0, 23.75, 27.0, 32.5]


In [ ]:
#df = pd.DataFrame({'A' :[1,1,3,2,6,2,8]})
#a = df['A'].unique()

In [58]:
result.dtypes
#So after parsing string '$##.##' to float we have
#Date which could be a date instead
#STart/End time which could be a time instead
#Miles which should be a float
#Duration which should be an int
#And Rest name and stacked which sort of need to be strings

ID                   int64
Date                object
Restaurant Name     object
Total              float64
Pay                float64
Tip                float64
Peak Bonus         float64
Stacked             object
Version            float64
Start Time          object
Miles               object
End Time            object
Est. Duration       object
dtype: object

In [90]:
#We can convert all dates, but start/end time is a v3 thing, so only convert when version == 3
datetest = result.iloc[600]["Date"] + "/2023"
timetest = result.iloc[600]["Start Time"]
#datetest = pd.Timestamp(datetest)
datetest = pd.to_datetime(datetest, format="%m/%d/%Y")
print(type(datetest))
print(datetest.strftime('%m-%d'), timetest)
#type(datetest)

#So maybe what we want to do, is convert date, start, end to datetime, exclude v1/v2 for start/end related analysis
#now try convert start time to date time
timetest1 = pd.to_datetime(timetest)
print(timetest1)

#interesting, it automatically uses the current date without an existing date
#maybe we just convert both to datetime, then use datetime.date for one and datetime.time for the other

#datetimetest = pd.Timestamp.combine(date(datetest), time(timetest1))
#print(datetimetest)
print(timetest1.hour, timetest1.minute, datetest.month, datetest.day)
datetimetest = pd.Timestamp.combine(date(datetest.year, datetest.month, datetest.day), time(timetest1.hour, timetest1.minute))
print(datetimetest)
print(type(datetimetest))

#Okay so that's how we can get the timestamp for date and for time, that leaves miles and duration to check

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
05-27 7:19:00 PM
2023-06-16 19:19:00
19 19 5 27
2023-05-27 19:19:00
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [94]:
#check miles and duration
a1 = result["Miles"].unique()
print(a1)
#print(sorted(a1))

#okay right, miles and duration only exist for v3, but we have nan and we have -
#I think I used - for when the data was v3 but unavailable

a2 = result["Est. Duration"].unique()
print(a2)

#So we have cleaned up the data, but we'll want to utilize two primary dataframes for deliveries
#one covering all versions (which newer attributes dropped)
#one covering v3 onward (which retains newer attributes)

[nan '6.2' '2.9' '5.5' '3.1' '3.3' '7.1' '4.84' '5.2' '3.7' '4.7' '3.0'
 '5.9' '5.1' '-' '8.2' '6.1' '7.2' '3.2' '2.6' '2.4' '9.5' '8.3' '2.3'
 '6.9' '4.5' '7.7' '2.5' '5.0' '1.8' '4.0' '7.3' '6.3' '5.3' '7.5' '2.0'
 '3.4' '3.6' '4.2' '6.0' '2.7' '7.4' '10.1' '8.0' '15.9' '7.6' '6.5' '4.4'
 '1.9' '4.3' '4.1' '1.4' '4.8' '1.2' '6.8' '10.2' '12.1']
[nan '30' '20' '28' '34' '25' '35' '15' '26' '18' '29' '32' '-' '41' '33'
 '21' '23' '17' '22' '37' '39' '19' '12' '52' '16' '27' '13' '14' '10']


In [95]:
DaysDF.head()

,Date,Total,Base,Tip,Start (24),End (24),Active time (h:m),Dash time (h:m),Offers,Deliveries,Version,Pauses
0,3/27,$28.75,$19.75,$9.00,11:53,14:06,1:19,2:12,4,4,1,1
1,3/31,$167.75,$88.75,$79.00,13:07,21:52,5:22,8:45,17,18,1,1
2,4/5,$36.00,$16.00,$20.00,14:12,15:38,1:04,1:26,4,4,1,1
3,4/6,$100.25,$48.75,$51.50,16:30,21:05,3:00,4:35,9,10,1,1
4,4/7,$197.94,$117.75,$80.19,17:16,1:31,5:54,8:15,20,24,1,1


In [96]:
WeeksDF.head()

,Dates,Base,Tip,Adj pay,Other,Pre Total,Total,Active,Dash,Deliveries,Version
0,3/27 - 4/2,$108.50,$88.00,$40.96,$0.00,$196.50,$237.46,6:42:00,10:55:00,22,1.0
1,4/3 - 4/9,$264.25,$230.69,$58.17,$0.00,$494.94,$553.11,14:41:00,22:02:00,55,1.0
2,4/10 - 4/16,$130.75,$96.13,$61.07,$0.00,$226.88,$287.95,8:38:00,11:10:00,28,1.0
3,4/17 - 4/23,$382.50,$248.41,$105.05,$0.00,$630.91,$735.96,21:20:00,32:18:00,68,1.0
4,4/24 - 4/30,$430.00,$299.73,$138.76,$2.62,$729.73,$871.11,25:30:00,41:10:00,84,1.0


In [114]:
#gpsDF.head()
gpsDF1 = gpsDF[["ID", "Date", "Restaurant Name", "RAW DATA"]].copy()
#gpsDF1.head()
gpsDF2 = gpsDF1[["Restaurant Name", "RAW DATA"]].copy()
#gpsDF2 = gpsDF2.drop_duplicates()
gpsDF2 = gpsDF2.drop_duplicates(subset=["Restaurant Name", "RAW DATA"])
gpsDF2

,Restaurant Name,RAW DATA
0,Erika's Restaurant,Erika's Mexican Food & Seafood Vista
1,Chick-fil-A,Chick-fil-A (01507)
2,Wendy's,Wendy's (8407)
3,Annam Pho and Sandwiches,Annam Pho and Sandwiches (Plaza Dr)
4,Disfruta,Disfruta (Oceanside)
...,...,...
688,Big Bob's Best Pizza,Big Bob's Best Pizza
696,???(tacobell)???,CCC0005121 (3845 Plaza Drive)
703,Del Taco,Del Taco (1305)
704,??(other taco bell),CCC0005121 (695 College Boulevard)


In [118]:
gpsDF2 = gpsDF2.sort_values(by="Restaurant Name", ascending=True)
gpsDF2
#okay neat, so in theory 204 restaurants visited
#lets throw this back out to a csv for ease of data entry
gpsDF2.to_csv('gpsDF2.csv')